In [ ]:
# pip install tensorflow==2.1.0 tfx==0.21.0

In [1]:
import os
import sys

import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from tfx.utils.dsl_utils import external_input
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen

In [2]:
# add project to the python path
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

os.environ["TFHUB_CACHE_DIR"] = '/Users/i854694/tmp/tfhub'

In [3]:
context = InteractiveContext(pipeline_root='../tfx')

In [4]:
base_dir = os.getcwd()
data_dir = "../data/"
examples = external_input(os.path.join(base_dir, data_dir))
example_gen = CsvExampleGen(input=examples)

context.run(example_gen)

ERROR:absl:mlmd client NotFoundError: No type found for query: tfx.components.example_gen.csv_example_gen.component.CsvExampleGen


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ../tfx/CsvExampleGen/examples/1, id: 2)]
        ))

In [ ]:
for artifact in example_gen.outputs['examples'].get():
    print(artifact)

In [5]:
from tfx.components.statistics_gen.component import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

context.show(statistics_gen.outputs['statistics'])

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-19T14_28_29.350030"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-19T14_28_29.350030"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-19T14_28_29.350030');
ERROR:absl:mlmd client NotFoundError: No type found for query: tfx.components.statistics_gen.component.StatisticsGen
/Users/i854694/.virtualenvs/pipelines/lib/python3.6/site-packages/tensorflow_data_validation/arrow/arrow_util.py:236: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [6]:
from tfx.components.schema_gen.component import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)

#context.show(schema_gen.outputs['schema'])

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-19T14_28_29.350030"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-19T14_28_29.350030"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-19T14_28_29.350030');
ERROR:absl:mlmd client NotFoundError: No type found for query: tfx.components.schema_gen.component.SchemaGen


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: ../tfx/SchemaGen/schema/3, id: 4)]
        ))

In [ ]:
context.show(schema_gen.outputs['schema'])

In [7]:
from tfx.components.example_validator.component import ExampleValidator

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-19T14_28_29.350030"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-19T14_28_29.350030"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-19T14_28_29.350030');
ERROR:absl:mlmd client NotFoundError: No type found for query: tfx.components.example_validator.component.ExampleValidator


,Anomaly short description,Anomaly long description
Feature name,,
'issue',Unexpected string values,"Examples contain values missing from the schema: Cash advance (<1%), Lender damaged or destroyed vehicle (<1%), Unexpected/Other fees (<1%)."


In [9]:
transform_file = os.path.join(base_dir, '../components/transform.py')

In [10]:
from tfx.components.transform.component import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=transform_file)
context.run(transform)

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-19T14_28_29.350030"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-19T14_28_29.350030"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-19T14_28_29.350030');
ERROR:absl:mlmd client NotFoundError: No type found for query: tfx.components.transform.component.Transform


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: ../tfx/Transform/transform_graph/5/.temp_path/tftransform_tmp/440ec8d94c704942a1ee6372323dcffa/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedMode

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: ../tfx/Transform/transform_graph/5, id: 6)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ../tfx/Transform/transformed_examples/5, id: 7)]
        ))

In [ ]:
# needs updating for 0.21.0

import pprint
import tensorflow_data_validation as tfdv

tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = transform.outputs['transformed_examples'].get()[1].uri

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a TFRecordDataset to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
decoder = tfdv.TFExampleDecoder()

# Iterate over the first 3 records and decode them using a TFExampleDecoder
for tfrecord in dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = decoder.decode(serialized_example)
    pp.pprint(example)

In [11]:
#trainer_file = os.path.join(base_dir, '../components/keras_trainer.py')

trainer_file = './keras_trainer.py'

In [12]:
from tfx.components.trainer.component import Trainer
from tfx.proto import trainer_pb2
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor

TRAINING_STEPS = 100
EVALUATION_STEPS = 100

trainer = Trainer(
    module_file=trainer_file,
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS))
context.run(trainer)

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-19T14_28_29.350030"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-19T14_28_29.350030"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-19T14_28_29.350030');
ERROR:absl:mlmd client NotFoundError: No type found for query: tfx.components.trainer.component.Trainer


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sub_product_xf (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
state_xf (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
zip_code_xf (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
company_xf (InputLayer)         [(None, 1)]          0                                            
______________________________________________________________________________________________

ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: ../tfx/Trainer/model/6, id: 8)]
        ))

In [ ]:
  model_analyzer = Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      eval_config=tfma.EvalConfig(
          model_specs=[tfma.ModelSpec(label_key='variety')],
          slicing_specs=[tfma.SlicingSpec(feature_keys=['sepal_length'])]))

In [18]:
import tensorflow_model_analysis as tfma
from tfx.components.evaluator.component import Evaluator

model_analyzer = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    eval_config=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='consumer_disputed')]))#,
        #slicing_specs=''))
context.run(model_analyzer)

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-19T14_28_29.350030"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-19T14_28_29.350030"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-19T14_28_29.350030');


	 [[{{node StatefulPartitionedCall/transform/transform/inputs/F_consumer_disputed/shape}}]] [Op:__inference_signature_wrapper_49996]

Function call stack:
signature_wrapper
. Attempting to run batch through serially.


RuntimeError: tensorflow.python.framework.errors_impl.InvalidArgumentError:  You must feed a value for placeholder tensor 'StatefulPartitionedCall/transform/transform/inputs/F_consumer_disputed/indices' with dtype int64 and shape [?,2]
	 [[{{node StatefulPartitionedCall/transform/transform/inputs/F_consumer_disputed/indices}}]] [Op:__inference_signature_wrapper_49996]

Function call stack:
signature_wrapper [while running 'ExtractEvaluateAndWriteResults/ExtractAndEvaluate/ExtractPredictions/Predict']

In [ ]:
from tfx.components.model_validator.component import ModelValidator

model_validator = ModelValidator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'])
context.run(model_validator)

In [ ]:
#mkdir

In [ ]:
!mkdir -p /tmp/serving_model_dir

from tfx.components.pusher.component import Pusher
from tfx.proto import pusher_pb2

_serving_model_dir = "/tmp/serving_model_dir"

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=model_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

In [ ]:
!cat /tmp/serving_model_dir/1580932079/assets/vocab_compute_and_apply_vocabulary_1_vocabulary